<pre>
Name : Rohan Mehta
Username : mehtaro
Email id: mehtaro@iu.edu
</pre>

# Final exam programming problem

In this problem, you will work on the following:

- For a graph G, you will compute its MST
- We will add an edge into the MST and you should be able to quickly update the MST

To solve this problem, you may consult three lab sessions:

- Merge sort (We will sort the edges using merge sort)
- MST (Kruskal's algorithm)
- DFS (To find a cycle on a graph)

In [257]:
import random

## Part 1

You must work with the provided data structures.
You may use your own data structures if they are not defined here.

In [258]:
# Define graph edge
class Edge:
    def __init__(self, node1, node2, weight=1.0):
        self.node1 = node1
        self.node2 = node2
        self.weight = weight
        
    def __lt__(self, other):
        selfPriority = self.weight
        otherPriority = other.weight
        return selfPriority < otherPriority

In [259]:
# Define an undirected graph
class UndirectedGraph:
    def __init__(self, n):
        self.num_nodes = n
        self.nodes = [set() for i in range(n)]
        self.edges = []
    
    # edge node1 <--> node2 (undirected)
    def insert(self, node1, node2, weight=1.0):
        self.nodes[node1].add(node2)
        self.nodes[node2].add(node1)
        self.edges.append(Edge(node1, node2, weight))
        self.edges.append(Edge(node2, node1, weight))

### Graph representation

- Linked list: self.nodes in class UndirectedGraph (for DFS)
- Edge list: self.edges in class UndirectedGraph (for MST)

In [260]:
# A function to generate a random graph
def generate_graph_and_output_edges(num_nodes, random_seed, num_edge_per_node=3):
    random.seed(random_seed)
    graph = UndirectedGraph(num_nodes)
    edge_list = []
    for i in range(num_nodes):
        while len(graph.nodes[i]) < num_edge_per_node:
            node = random.randint(0, num_nodes-1)
            if node != i and (node not in graph.nodes[i]):
                weight = random.random()
                graph.insert(i, node, weight)
    return graph  

### A small sample graph

This is a sample graph so you can learn how graph is represented in this problem

In [261]:
# Generate the graph
graph_sample = generate_graph_and_output_edges(5, 100, 2)

# print nodes represented in linked list
print(graph_sample.nodes)

# print edges represented in edge list
for e in graph_sample.edges:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

[{1, 2, 3}, {0, 3, 4}, {0, 3, 4}, {0, 1, 2}, {1, 2}]
 0,  1, 0.45953
 1,  0, 0.45953
 0,  3, 0.73196
 3,  0, 0.73196
 1,  3, 0.50687
 3,  1, 0.50687
 2,  0, 0.53290
 0,  2, 0.53290
 2,  3, 0.26342
 3,  2, 0.26342
 4,  1, 0.33535
 1,  4, 0.33535
 4,  2, 0.83890
 2,  4, 0.83890


In [262]:
# Union Find data structure is provided to support Kruskal
class UnionFind:
    def __init__(self, num_nodes):
        # Initially position[i] = i
        self.position = [i for i in range(num_nodes)]
        
    # Return the cluster index
    def find(self, node):
        if self.position[node] == node:
            return node
        else:
            self.position[node] = self.find(self.position[node])
            return self.position[node]
    
    def union(self, node1, node2):
        a = self.find(node1)
        b = self.find(node2)
        # no need to union
        if a == b:
            return
        # union is needed
        else:
            if a < b:
                self.position[b] = a
            else:
                self.position[a] = b

## Part 2

Find a MST for a given graph.

The returned MST should be in edge list format. (Just like what we did in the lab)

When sorting the graph.edges, you must use Merge Sort. (We used PriorityQueue in the lab)

In [263]:
# Implement MergeSort

# Your merge sort should returend a sorted list of A
def merge(X, Y):
    m = len(X)
    n = len(Y)  
    i = 0
    j = 0
    result = []
    while(i < m and j < n):
        if(X[i] < Y[j]):
            result.append(X[i])
            i = i + 1
        else:
            result.append(Y[j])
            j = j + 1
    if(i < m):
        while(i < m):
            result.append(X[i])
            i = i+1
    if(j < n):
        while(j < n):
            result.append(Y[j])
            j = j + 1
    return result
                
    
def MergeSort(A):
    num = len(A)
    '''
    
    fill in the code
    
    '''
    if(num == 1):
        return A
    mid = num // 2
    left = MergeSort(A[:mid])
    right = MergeSort(A[mid:])
    return merge(left, right)

In [264]:
# Implement Kruskal

# returned MST is a list of class Edge
def kruskal(graph):
    MST = []
    uf = UnionFind(graph.num_nodes)

    sorted_edge_list = MergeSort(graph.edges)
    '''
    
    fill in the code
    
    '''
    size = graph.num_nodes - 1
    while((len(MST)!=size) and (len(sorted_edge_list)>0)):
        sorted_e = sorted_edge_list.pop(0)
        s_node1 = sorted_e.node1
        s_node2 = sorted_e.node2
        uf_a = uf.find(uf.position[s_node1])
        uf_b = uf.find(uf.position[s_node2])
        if(uf_a != uf_b):
            MST.append(sorted_e)
            uf.union(uf_a, uf_b)
    
    return MST

## Part 3

A new edge (2, 19) with weight 0.2

We need to quickly update the old MST to get a new MST

In [265]:
# Construct a graph from edge list. This is given to you.

# We assume num_nodes is given for simplicity.
def generate_graph_from_edges(edge_list, num_nodes):
    graph = UndirectedGraph(num_nodes)
    for e in edge_list:
        graph.insert(e.node1, e.node2, e.weight)
    return graph

### Algorithm

- We add the new edge to the old MST which will generate a cycle

- The cycle consists at least three edges

- The edge with the largest weight should be removed

- You ONLY need to print out the edges on the cycle to get full marks (the two end nodes and the weight)

### Sample example

- We have a MST for a graph of 3 nodes

- MST has edge (0, 1, 0.12) and (0, 2, 0.23)

- We add a new edge (1, 2, 0.2)

- A cycle of three edges is generated

- We need to delete edge (0, 2, 0.23)

### How to use DFS to find the cycle

- Suppose the new edge is (2, 19, 0.2)

- If we DFS from node 2, we will eventurally hit node 19

- You need to properly append and pop on "cycle_edges" during DFS 

- The "cycle_edge" should contain the final path from 2 to 19

In [266]:
# Use DFS to detect the cycle

# You may add more arguments in dfs_recursive() for your convenience

def dfs(graph, starting_node, stopping_node):
    visited_nodes = [False for i in range(graph.num_nodes)]
    cycle_edges = []
        
    dfs_recursive(graph, starting_node, visited_nodes, cycle_edges, stopping_node)
    new_cycle_edges = backtrack(cycle_edges,starting_node,stopping_node)
    # The following is for demonstration purpose
    print("We found {:d} edges from node {:d} to node {:d}"
          .format(len(new_cycle_edges), starting_node, stopping_node))
    for e in new_cycle_edges:
        '''
        print the two end nodes of the edge
            the weight information is not known so you do not need to output 
        
        You may change the following print statement
            if you did not use Edge class
        '''
        #print("{:2d}, {:2d}".format(e.node1, e.node2))
        print("{:2d}, {:2d}".format(e[0], e[1]))


def dfs_recursive(graph, starting_node, visited_nodes, cycle_edges, stopping_node):
    
    if(starting_node == stopping_node):
        return
    visited_nodes[starting_node] = True
    if(len(graph.nodes[starting_node]) > 0):
        for i in graph.nodes[starting_node]:
            if(visited_nodes[i] == False):
                cycle_edges.append((starting_node,i))
                dfs_recursive(graph,i,visited_nodes,cycle_edges,stopping_node)

In [267]:
def backtrack(A,start,stop):
    backtrack = []
    for i in range(len(A)):
        if(A[i][1] == stop):
            initial = A[i]
        
    backtrack.append(initial)
    num = len(A)-2
    while(num >= 0):
            
        if(A[num][1] == initial[0]):
            initial = A[num]
            backtrack.append(initial)
            num = num-1
        else:
            num = num-1
    backtrack.reverse()
    return backtrack
            

## Part 4 Test code

Do NOT modify the code in this part.


In [268]:
graph_mst = generate_graph_and_output_edges(20, 10, 5)
MST = kruskal(graph_mst)

for e in MST:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

18,  3, 0.00406
 1,  5, 0.03175
18,  0, 0.03259
 5,  0, 0.03440
11,  1, 0.04456
 2, 18, 0.05057
15, 13, 0.06277
15,  4, 0.06499
 7, 11, 0.07993
 4, 19, 0.10876
 8,  9, 0.14351
16,  5, 0.15642
 9, 10, 0.16494
14,  7, 0.16636
16,  9, 0.17846
 4,  3, 0.19495
 4,  6, 0.38442
 5, 12, 0.39059
 0, 17, 0.43858


In [269]:
def find_the_cycle(MST, new_edge, num_nodes):
    # construct graph from edge list
    graph_cycle = generate_graph_from_edges(MST, num_nodes)

    # call dfs to find a path from node1 to node2
    dfs(graph_cycle, new_edge.node1, new_edge.node2)

In [270]:
find_the_cycle(MST, Edge(2, 19, 0.2), 20)

We found 4 edges from node 2 to node 19
 2, 18
18,  3
 3,  4
 4, 19


In [271]:
# Instead we can add edge (3, 14, 0.1)
find_the_cycle(MST, Edge(3, 14, 0.1), 20)

We found 7 edges from node 3 to node 14
 3, 18
18,  0
 0,  5
 5,  1
 1, 11
11,  7
 7, 14


### Hint

If you run the following codes,

In [272]:
graph_hint = generate_graph_and_output_edges(10, 1, 2)

mst = kruskal(graph_hint)

print("The minimum spanning tree:")
for e in mst:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

find_the_cycle(mst, Edge(1, 8, 0.2), 10)

The minimum spanning tree:
 8,  6, 0.00920
 0,  5, 0.02232
 1,  0, 0.25507
 6,  3, 0.43277
 1,  3, 0.48786
 0,  9, 0.52763
 2,  0, 0.56920
 3,  4, 0.59115
 7,  2, 0.65159
We found 3 edges from node 1 to node 8
 1,  3
 3,  6
 6,  8


In [273]:
# this is the results you should get:
'''
The minimum spanning tree:
 8,  6, 0.00920
 0,  5, 0.02232
 1,  0, 0.25507
 6,  3, 0.43277
 1,  3, 0.48786
 0,  9, 0.52763
 2,  0, 0.56920
 3,  4, 0.59115
 7,  2, 0.65159
We found 3 edges from node 1 to node 8
 1,  3
 3,  6
 6,  8
'''
print()

## Programming Problem 2
A modified knapsack problem is that for a given set of positive integers {a1, a2, ..., an}, a knap- sack of size s, find a subset A of {a1, a2, ..., an} such that the sum of elements in A is the largest, but at most s. Write a program using Python to solve this problem using a top down dynamic programming method.
Test your program for the following modified knapsack problem:
Input list: {5, 23, 27, 37, 48, 51, 63, 67, 71, 75, 70, 83, 889, 91, 101, 112, 121, 132, 137, 141, 143, 147, 153, 159, 171, 181, 190, 191} with knapsack size s = 762. Print out a subset along with the sum of its elements so that the sum has the closest distance to 762.

In [274]:
def knapsack(A,s):
    result = []
    comp = [[0 for i in range(s + 1)] for j in range(len(A) + 1)]
    
    for i in range(len(A)+1):
        for j in range(s+1):
            if(i == 0 or j == 0):
                comp[i][j] = 0
            elif(A[i-1] <= j):
                comp[i][j] = max(A[i-1]+comp[i-1][j-A[i-1]],comp[i-1][j])
            else:
                comp[i][j] = comp[i-1][j]
                
    chain = comp[i][j]
    size = len(comp)-1
    while size >= 0:
        num = size
        if(chain in comp[size]):
            while num >= 0:
                if(chain in comp[num]):
                    num = num - 1
                else:
                    size = num + 1
                    break
            
            key = size - 1
            chain = chain - A[key]
            result.append(A[key])
            size = size - 1
        else:
            size = size - 1
    result.pop(len(result) - 1)
    print ("The subset is:", result)
    print("The sum of off the elements in the subset is:", comp[i][j])
    return result


if __name__ == "__main__":
    A =  [5, 23, 27, 37, 48, 51, 63, 67, 71, 75, 70, 83, 889, 91, 101, 112, 
          121, 132, 137, 141, 143, 147, 153, 159, 171, 181, 190, 191]
    s = 762
    knapsack(A, s)

The subset is: [101, 91, 83, 70, 75, 71, 67, 63, 51, 48, 37, 5]
The sum of off the elements in the subset is: 762


# Multiple Choice Questions

### Problem 1.1 
#### Consider an undirected graph G = (V, E) with edge costs that are all positive and distinct. Let T be a minimum spanning tree for graph G. Let P be a shortest path from s to t, where s, t ∈ V. Which of the following statements is correct?

### Ans. (A) Suppose each edge weight is increased by 1, the minimum spanning tree T will not change

### Problem 1.2 
#### Which of the following statements about a tree of n nodes is wrong?

### Ans. (B) Removing the root node is the only way to disconnect the tree

### Problem 1.3 
#### Which of the following statements about a Binary Search Tree with distinct n elements is wrong?

### Ans. (B) The smallest element is always at the leaf of the tree

### Problem 1.4
#### The algorithm in concern has a time complexity of O(n2). The algorithm takes 40 seconds to run with input size 10. How much time it will require to run with input size 30?

### Ans. (D) 6 minutes

### Problem 1.5 
#### Consider the following unsorted array of strings, R = {cat, him, ham, bat}. What will be the order of elements of R after the first iteration of LSD Radix sort if we want to sort the array in an ascending order?

### Ans. (B) {him, ham, cat, bat}

### Problem 1.6 
#### Using Brute-Force pattern matching algorithm, which pattern below will give you the fastest match with the given string “abacaabaccabacabaabb"?

### Ans. (D) bacca

# Non-multi Choice Questions

## Problem 2
### Let G = (V, E) be a connected directed graph with non-negative edge weights. Let s and t be vertices of G. We delete c (not a constant number) edges in G to get a subgraph H. Suppose we want to add back exactly one deleted edge into H, so that the shortest path from s to t in the resulting graph is as short as possible.
### Describe an algorithm that chooses the best edge to reinsert. Your algorithm should run in O(m log n) time in which m is the number of edges and n is the number of vertices in G.
#### Hint 1: Dijkstra’s algorithm runs in O(m log n) time on graph G.
#### Hint 2: Number of deleted edges c is not a constant number, so running Dijkstra’s algorithm c times will result in O(c m log n) running time.

### Ans)
<pre>
It looks that Dijkstra's algorithm approach will be necessary, based on the question and the suggestions that have been 
provided. We are unable to perform Dijkstra's for all edges that have been deleted from the graph G and then 
choose the one that provides the shortest path since this would need "c" Dijkstra's iterations. 
 
There is one approach that can be taken into consideration.

Instead of offering the shortest path between specific nodes "a" and "b," we modify Dijkstra's approach such that 
it returns the shortest paths between all nodes except "a".
  
Now we run Dijkstra's twice for each "s" and "t". Assuming that "s" is the source and that Forward(x<sub>i</sub>) is the shortest path to node x<sub>i</sub>.

We then repeat Dijkstra's algorithm with "t" as the source and all edges reversed, and the shortest path to node y<sub>i</sub> is referred to as Backward(y<sub>i</sub>).

Now, we remove "c" edges from graph G, which are C ={(x<sub>1</sub>, y<sub>1</sub>), (x<sub>2</sub>,y<sub>2</sub>),
...(x<sub>c</sub>,y<sub>c</sub>)}.  

We look for an "i" such that the sum of Forward(x<sub>i</sub>) + edge(x<sub>i</sub>, y<sub>i</sub>) + Backward(y<sub>i</sub>) is  minimum, where x<sub>i</sub>, y<sub>i</sub> belongs to C. The edges x<sub>i</sub>, y<sub>i</sub> are the best edges to reinsert in this case. 

The algorithm has a runtime of (2mlogn + c + m), which is asymptotically O(mlogn). The following components are described in further detail: 2mlogn -> Two iterations of Dijkstra's algorithm in its variant form.
We have taken: 
c -> Trying out all of the edges that have been removed to see which ones work best to put back in
m -> To reverse all edges, it will be required to calculate Backward.
</pre>


## Problem 3
### Let G = (V, E) be a connected undirected graph with m edges and n nodes. If we remove a node v from G, we get a subgraph Gv. Given a graph G and a node v, how can you know if subgraph Gv is connected or not? Please describe a O(m + n) algorithm to decide the connectivity of Gv .

### Ans)
<pre>
Starting at any node in the subgraph and applying DFS or BFS while retaining the visited node 
will allow us to determine whether or not the subgraph is connected. If the total number of nodes 
is equal to the total number of nodes that have been visited, then the graph is said to be linked. 
Furthermore, the worst-case complexity is O(m+n) because it is either BFS or DFS, indicating that it is O(m+n).

-> Algorithm is as follows:

1. add edges and creat undirected graph:
dict = {} to store the graph
def addedge(a,b): # for adding the new node which was not in the graph before

    if new_element not in dict:
        dict[a] = []
    add b to dict[a]
  
2. Apply dfs function :
create a list for visited nodes
    visited = []  # keeps track of all the visited nodes
    
    def dfs(x): 
        visited[x] = true # if the node is in the dictionary it will make the bollean value true for the node
        
        for i in dict[x]: # i will be the next connected node from the current node
        
            if i in visited[]:
                i++ 
                
            else: 
                dfs(i) # call the function recursively and make the boolean value True for the nodes

3. Check the visited nodes :

    if visited[x] is false: # checking if all the nodes are present in visited list(having True value)
        Not Connected
        
    else:
        Connected
</pre>

## Problem 4
### A long string (10,000 characters) consists of four characters: A, T, G, C; they appear with frequency 1/2, 1/4, 1/8 and 1/8, respectively.
#### (a) What is the Huffman encoding for each character in the alphabet? Draw the Huffman encoding tree for each step of the algorithm. The final Huffman encoding tree should look like the following tree. You may draw by hand and upload a picture.
#### (b) Originally each character is represented in two bits. How many bits are saved if we use the new encoding scheme we obtained in the previous question?

### Ans)
(A)
<pre>

----------------------- <br>
|Letter     |Frequency| <br>
| A         | 1/2     | <br>
| T         | 1/4     | <br>
| G         | 1/8     | <br>
| C         | 1/8     | <br>

----------------------- <br>
<br>

Step 1: Adding the two alphabets with lowest frequency, i.e. G & C in our case with 1/8 frequency.

                                           1/4
                                          /   \
                                         0     1
                                        /       \
                                       G         C 
                                      1/8       1/8 
 
 Step 2: Adding the alphabet T with the 2nd lowest frequency, in our case with 1/4 frequency.

                                         1/2
                                         /  \
                                        /    1
                                       /      \
                                      0       1/4
                                     /       /   \
                                    /       0     1
                                   /       /       \
                                  T       G         C 
                                 1/4     1/8       1/8 

Step 3: Adding the alphabet A with the highest frequency, i.e. in our case with 1/2 frequency.

                                          1   
                                         / \ 
                                        /   1 
                                       /     \
                                      /     1/2
                                     /      /  \
                                    0      /    1
                                   /      /      \
                                  /      0       1/4
                                 /      /       /   \
                                /      /       0     1
                               /      /       /       \
                              A      T       G         C 
                             1/2    1/4     1/8       1/8 
                           
As mention in Huffman encoding we have labelled all the left nodes with 0 and all the right nodes with 1. 
Finally, the result is : A = 0, T = 10, G = 110, C = 111.
     
You can see the huffmans tree we were intended to create in the image above. The tree was included in the exam's pdf file. As tiny as feasible while being conveyed, the primary purpose of the huffmans encoding algorithm is to keep a message's bit count low. 1/2, 1/4, 1/8, and 1/8 of the 10000 characters in the lengthy string are utilized. Each letter in the alphabet has four bit codes: A, T, G, and C. The numbers are 0, 10, 110, and 111. The bit count will be 5000, 5000, 3750, 3750 respectively, it'll contain 17500 bits in total. To this list will be added the ASCII character count of 8 and a coding extension of 9 bits. After that, the code will be 17-bit long. Bits have been exchanged at a rate of 17517 per second. In total, 17517 bits are sent.

(B) Without using Huffmans encoding , the total bits transferred will be 10000*2 that will be 20000 bits in this transmission. By the usage of Huffmans encoding we save approximately 2483 bits. 
</pre>

## Problem 5
### Let G = (V,E) be a undirected graph with m edges and n nodes; and let T = (V,E′) be its minimum spanning tree. All edge weights are distinct. Assume we remove an edge (u, v) ∈ E′ from graph G. Describe an algorithm to update the minimum spanning tree in O(m) time.
#### 1. Hint 1: The removal of edge (u, v) disconnects the spanning tree T in to two set of nodes A and B. Without a loss of generality, we may assume u ∈ A and v ∈ B.
#### 2. Hint 2: We need to find the smallest weight edge whose one end node in A and the other end node in B.
#### 3. Hint 3: In order to get full marks, you need to state clearly what algorithm you use to find set A and B; what data structure you use to efficiently look up for target edge; and what are the running time of each portion of your algorithm.

### Ans)
<pre>
Let us assume we have a Graph G. T is the MSP of that Graph G. Since T is the MST, we know that all minimum weight edges are used in the MST. 
Let us remove an edge (u,v) which belongs to T (our MSP). We need to find a new edge (u',v') such that it builds our new MSP.
Assume that whenever we mention an edge, we are storing the values u, v, and w.
By already knowing that removal of the edge disconnects our T. Let the two Graphs formed be D1 and D2.
The first step is to create two HashSets S1 and S2 which stores vertices of the 2 disconnected graphs D1 and D2.  (Set in python)
We're constructing two hashSets to make it easier to identify a vertex in each graph, rather than having to loop over each graph to get the vertices.
The following step is to establish a MinHeap M, which will store all of the edges of G in a single location (our original graph which does not contain the deleted edge).
To do this task, we just need to pop edges from our MinHeap M and determine whether or not the edge contains one vertex from D1 and the other vertex from D2.
Because we are popping elements from MinHeap M, we are certain to discover the edges with the lowest weight, and the first edge that links D1 and D2 will replace the edge that was removed, resulting in the formation of our new MST.

The time complexity of the Algo is O(m) where m = number of edges. We loop through all the edges and store them in MinHeap M.
Popping elements from MinHeap takes O(1) time and checking vertices of popped edge takes O(1) time since we are using HashSets.
</pre>

## Problem 6
### We would like to solve word search puzzle problem in which we are given a N by N character board and N words. In the word search puzzle, the N words may be placed horizontally, vertically, or diagonally in the world puzzle board. Your task is to find all the N words. 
### If you are not familiar with word search puzzle, please refer to https://thewordsearch. com for more details. 
### Please describe a O(N^3) algorithm to solve this problem. You need to specify what data structure you use and what are the time complexity at each step.

### Ans)
<pre>
Due to the fact that the subject includes matching words, the Trie data structure is an excellent choice for this task. It is possible to verify whether a given prefix matches the prefix of any of the words recorded in the Trie data structure using an upgraded Trie data structure.
Algorithm:  
1. Fill all the N words in a Trie.
2. To begin with, apply DFS search at every possible location on the board.
   1. Continue DFS search if only for as long as there is prefix match in the Trie.
   2. Store locations of found words in case of complete word match.
   3. Stop DFS search if there is no prefix match.
Assume "n" is the average length of N words.  
Runtime analysis: DFS for a given location takes 8<sup>n</sup> time. We do DFS on the entire board, so N<sup>2</sup>8<sup>n</sup>. Do this for all the N words, N<sup>3</sup>8<sup>n</sup>. If "n" is very small compared to N and 8<sup>n</sup> can be assumed to be a constant. This is O(N<sup>3</sup>).  
So the algorithm is O(N<sup>3</sup>), based on the assumption above.
</pre>

## Problem 7
### What is the time complexity of the function calc in big-Oh? What is the return value k in terms of big-Oh notion? For full credit please show your work step by step.
<pre>
def calc(n): 
    k=0
    i = n/2 
    while(i<=n):
        i=i+1 
        j=2 
        while(j<=n):
            j=j*2
            k = k + n/2
    return k
</pre>

### Ans) 
<pre>
Due to the fact that the initial glance takes N/2 time, which is linear in time, the prior strategy requires O(N) iterations to complete the first loop before moving on to the next. The second loop iterates over and over again, exponentially repeating the first. It takes N/2logn times for the process to be completed since the value of k increases by N/2 with each iteration. To put it another way, the complete procedure has a temporal complexity of O(N*log(N)).
</pre>

## Problem 8
### What are the advantages of QuickSort over MergeSort? If we want to sort a linked list of elements, which one between them should we choose? Please explain your answer.

### Ans) 
### Advantages of Quicksort over MergeSort: 
- You don't have to partition the array of components into equal halves before you start sorting. There is no limit to the number of equal portions that the array can be split into during quick sort. 
- Quick sorting is advantageous because it does not necessitate a lot of extra space because quicksort uses main memory, whereas merge sort requires extra memory(auxiliary memory) to perform the same task. 
- Quicksort outruns Mergesort in terms of speed because the results of quicksort are relatively close to each other in the cache, so it is much faster than merge sort.
- Quick sort is more efficient and faster than merge sort when working with tiny arrays or datasets in terms of both time and money. 
- When it comes to memory regions that can be used repeatedly, Quick sort outperforms Merge sort. 
- Locality of reference, i.e. a processor's proclivity to repeatedly access the same memory regions in a short amount of time is good when using quicksort unlike mergesort. 
### For sorting a linked list of elements Merge sort should be used over Quick sort for the following reasons:
- It runs in O(nlogn) time even in the worst case,on the contrary quick sort runs in O(n2) in worst case. 
- It's stable and Quick sort is not
- It doesn't need a lot of extra memory, de-facto it uses a constant amount of auxilary memory while computing linked list and quicksorts in-place memory usage doesn't count as an advantage for this case.
- As a result, merge sort performs more "short range" operations on the data, making it a better match for linked lists, whereas quicksort is a better fit for random access data structures (such as hash tables).

## Problem 9
### Consider a pattern matching problem with string s = “GCATGACTGCGTGACC" and pattern p= “CTGC". Using the Boyer-Moore algorithm find the lowest index i within s at which the matched (with p) substring begins. Follow the simplified Boyer-Moore algorithm step by step to find the value of i as well as demonstrate how many comparisons are made.

### Ans)
<pre>
We are given the following string, s = “GCATGACTGCGTGACC" 
and we have to find the pattern, p= “CTGC" from the given string s. 
We are supposed to use boyer-moore algorithm to find the pattern.


    | Bad Character Table |   |   |       |
    |---------------------|---|---|-------|
    | C                   | T | G | other |
    | 0                   | 2 | 1 | 4     |
    
    
By using the bad character table in boyer-moore algorithm, we will do the following steps:

GCATGACTGCGTGACC
CTGC
GCATGACTGCGTGACC
-----CTGC
GCATGACTGCGTGACC
----------CTGC

As a first stage, we are looking for a pattern in the string by going from right to left, i.e. starting with index 3. Due to the fact that it cannot detect any matching characters, it will go to the matching index, which is the index 6th. Following the discovery of the matching letter C, it will proceed to examine the other character G in the string. As the character A does not appear in the pattern, the pattern will skip the entire character until after A. Following that, it will go to the 9th index, where it will look for the pattern in the string. A total of three steps were required to locate the string in Boyer Moore's bad character method.

Using the good suffix method in boyer moore we will do the following steps:

GCATGACTGCGTGACC
CTGC
t = _

GCATGACTGCGTGACC
-----CTGC
t = C

GCATGACTGCGTGACC
----------CTGC
t = CTGC

As the good suffix method works same as the bad suffix method in the following example it can be skipped due to the equivalent effeciency.
</pre>

## Problem 10
### Consider an array of n positive integers, A = {a1, a2, ..., an} and a positive integer m. Using a top down dynamic programming method, design an algorithms that answers us if some subset of A add up to m where you can use each an atmost once.
### For example, a list of 5 positive integers A = {3, 7, 11, 15, 21} and the given integer m = 25. Your designed problem will answer True as a subset of A, {3, 7, 15}, sums to the given value of m.
### Prove the correctness of your algorithm and show the computation time with regards to big-Oh notion.

### Ans) 
<pre> In my view that HashMap is the most efficient method of solving this problem since it allows us to store both 
the current total and its count. One efficient method of traversing the array is to save the total sum up to this point in a variable called current sum. In addition, we count the number of different current sum values that exist in a map. At every point in time, if the current sum meets the required sum (in our example, m), the count of subarrays is increased by 1. When current sum – sum is employed, the value of current sum surpasses the value of the intended sum. It is possible to obtain the required sum by subtracting this number from the existing sum.
The map allows us to count the number of subarrays that have a sum that is the same as the current sum minus the 
current sum. In order to construct new subarrays with the needed total, all of the subarrays in the present
subarray must be removed from it. As a result, increase the number of such subarrays by the number of such 
subarrays. As soon as the current total meets the needed sum, count the number of subarrays that had a prior 
value of zero. Subarrays with the requisite total number of elements are formed when such subarrays are removed
from a current subarray.

<b>Algorithm </b>


def sumsubarray(array, m, n): # here m = the given integer, array is the dataset of integers, 
                                and n is the length of array 
    prefixsum = dict() # using this dictionary as a hashmap for storing all the different sum 
                        values from index 0 of the array while traversing through each element 
    current_sum = 0 # total sum of elements so far 
    totalnum = 0 # count of number of subarrays in the array whose sum is equal to k 
    for i in range(0,n): 
        current_sum = current_sum + ith element of array 
        
        if current_sum == m: # check if the current sum value matches m 
            result = result + 1 # increment result by 1 
        else if (current_sum - m) in prefixsum: # check if the subtraction of m from current sum is 
                                                    already in the hashmap 
            result = result + prefixsum[current_sum - m] 
         
         prefixsum[current_sum] = prefixsum[current_sum] + 1 #increasing the count of a particular sum if it 
                                                    repeats while traversing through the array return result 
<pre>

## Problem 11
#### Consider the following completed dynamic programming table, L, for a Longest Common Subsequence problem for string X with length 10 and string Y with length 12.
#### • What is the length of the longest common subsequence? Explain. Answer this question on the perspective of just by looking at the values of the table L.
#### • Show the path, in L(i, j), that a tracking algorithm will take to recover the longest common subsequence starting from L(9, 11). Given: the characters of X with indices {4, 5, 6, 7, 8, 9} match with the characters of Y with indices {0, 3, 4, 5, 7, 11}.


### Ans)
<pre>
a.) By observing the completed dynamic programming table, the bottom right corner of the table 
pertains to L([length_X, length_Y]). By looking at this we can conclude that the length of the 
Longest Common Subsequence is 6. The following algorithm is used to fill the given Dynamic programming 
table is given below: 

if(X[i] = Y[j]): 
    L[i,j] = 1 + L[i-1,j-1] 
else: 
    L[i,j] = max(L[i-1,j] , L[i,j-1]) 
    
Since, the largest number in the table is 6, therefore, the length of LCS is 6. 

b.) Here, we have given the characters of X with indices {4,5,6,7,8,9} match with characters of 
Y with indices {0,3,4,5,7,11}. The traversal logic given below is used to track the path.

1] For each and every time there is a match (X[i − 1] == Y [j − 1]), we include that character as 
a part of LCS and move to L[i − 1][j − 1] cell i.e we move upwards diagonally. 

2] On the other hand if there is a mismatch, we compare value of L[i][j] with L[i-1][j] and L[j][i-1] 
and will go in the direction of greater values. 

Therefore, the path followed is:
(9,11)   Here X[9] == Y[11] This means i-1 and j-1  -> this character X[i-1] is pushed to common substring.  
(8,10)   Here X and Y do not match and L(8,9) >> L(7,10)  
(8,9)    Here X and Y do not match and L(8,8) >> L(7,9)  
(8,8)    Here X and Y do not match and L(8,7) >> L(7,8)  
(8,7)    Here X[8] == Y[7] This means i-1 and j-1  -> this character X[i-1] is pushed to common substring.  
(7,6)    Here X and Y do not match and L(7,5) >> L(6,6)  
(7,5)    Here X[7] == Y[5] This means i-1 and j-1  -> this character X[i-1] is pushed to common substring.  
(6,4)    Here X[6] == Y[4] This means i-1 and j-1  -> this character X[i-1] is pushed to common substring.  
(5,3)    Here X[5] == Y[3] This means i-1 and j-1  -> this character X[i-1] is pushed to common substring.  
(4,2)    Here X and Y do not match. 
However, the algorithm determines that both (4,1) and (3,2) are equal in the else condition, therefore it will exit. If we implement the code as shown below, we have a 50-50 probability of getting the proper solution at this stage if one of the two is chosen without foreknowledge. We decided to move ahead y such that the following iteration would include j-1.
(4,1)    Here X and Y do not match and L(4,0) = L(3,1). using the same movement as before we move to j-1.    
(4,0)    Here X[4] == Y[0] This means i-1 and j-1  -> this character X[i-1] is pushed to common substring.  
(4,-1)   Exit condition reached.  

Hence, the path that would be taken by the tracking algorithm to recover the LCS is: <br>
<b> L(9,11)-> L(8,10)-> L(8,9) -> L(8,8) -> L(8,7) -> L(7,6) -> L(7,5) -> L(6,4) -> L(5,3) -> L(4,2) -> L(4,1) 
->L(4,0) -> L(3,-1). <b>
<pre>